In [1]:
# Import needed packages

import pandas as pd
import os

In [2]:
# Go to directory folder where our dataset exists

path="/Users/student/Desktop/SpringBoardDSCareerTrack/Capstone 2 Datasets/Vehicle Loan Payments"
os.chdir(path)
os.getcwd()

'/Users/student/Desktop/SpringBoardDSCareerTrack/Capstone 2 Datasets/Vehicle Loan Payments'

In [3]:
# Import data as a pandas dataframe

df = pd.read_csv('test.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112392 entries, 0 to 112391
Data columns (total 40 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UniqueID                             112392 non-null  int64  
 1   disbursed_amount                     112392 non-null  int64  
 2   asset_cost                           112392 non-null  int64  
 3   ltv                                  112392 non-null  float64
 4   branch_id                            112392 non-null  int64  
 5   supplier_id                          112392 non-null  int64  
 6   manufacturer_id                      112392 non-null  int64  
 7   Current_pincode_ID                   112392 non-null  int64  
 8   Date.of.Birth                        112392 non-null  object 
 9   Employment.Type                      108949 non-null  object 
 10  DisbursalDate                        112392 non-null  object 
 11  State_ID     